In [297]:
### Importing relevent lib ###
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
import time
import pandas as pd
import bs4
from bs4 import BeautifulSoup 
import requests
import scipy as sc
import numpy as np
import re


In [298]:
### get all genre links to scrapt information of each genre Anime in myAnimeList ###
def get_all_genre_links(site_url,genre_dic):
    genre_url = []
    for i in genre_dict:
        genre_url.append(site_url + genre_dict[i]+ '/'+i)
    return genre_url

### All Genres in Site ###
genre_dict = {
    'Action': '1',
    'Adventure':'2',
    'Cars':'3',
    'Comedy':'4',
    'Dementia':'5',
    'Demons':'6',
    'Drama':'8',
    'Ecchi':'9',
    'Fantasy':'10',
    'Game':'11',
    'Historical':'13',
    'Horror':'14',
    'Kids':'15',
    'Harem':'35',
    'Josei':'43'
}

In [299]:
### get all top anime links ###
### Sample works on 10,000 diffrent anime ###
def get_all_top_links(site_url):
    all_top_urls = []
    for i in range(0,10000,50):
        all_top_urls.append(site_url+'?limit='+str(i))
    return all_top_urls

In [300]:
### Load a soup Object ###
def load_soup_obj(html):
    html_get = requests.get(html).text
    soup = BeautifulSoup(html_get,'lxml')
    return sou

In [301]:
### Site's url's ###
mal_genre_front_page = 'https://myanimelist.net/anime.php'
top_anime_url = 'https://myanimelist.net/topanime.php'
### All genre Links list ###
mal_genre_links = get_all_genre_links(mal_genre_front_page,genre_dict)
### All Top 0-19,000 Anime's in mal URL list ###
top_anime_list = get_all_top_links(top_anime_url)


In [302]:
### Engline The Episodes ###
def parse_episodes(listt):
    result = []
    for i in listt[:3]:
        r = i.strip()
#         print(r)
        result.append(r)
    return result

In [352]:
def delete_ep_string(list_ep):
    
    
    res = []

    for i,_ in enumerate(list_ep):
        search_TV = re.search('TV',list_ep[i])
        search_OVA = re.search('OVA',list_ep[i])
        search_Movie = re.search('Movie',list_ep[i])
        if search_TV:
            a = list_ep[i].split('TV')[1]
            res.append(a)
        elif search_OVA:
            a = list_ep[i].split('OVA')[1]
            res.append(a)
        elif search_Movie:
            a = list_ep[i].split('Movie')[1]
            res.append(a)
        else:
            res.append(np.nan)
            continue
    return res


In [353]:
def delete_members_string(list_mem):
    
    res = []
    
    for i,_ in enumerate(list_mem):
        search = re.search('members',list_mem[i])
        if search:
            a = list_mem[i].split('members')[0]
            res.append(a)    
    return res

In [396]:
def split_dates(list_dates):
    result_s = []
    result_f = []
    
    
    for i,_ in enumerate(list_dates):
        res_s = list_dates[i].split('-')[0]
        res_f = list_dates[i].split('-')[1]
        result_s.append(res_s)
        result_f.append(res_f)
    
    result = {'Start':result_s,'Finish':result_f}
    return result
    

In [430]:
def split_type(list_ep):
    
    
    res = []

    for i,_ in enumerate(list_ep):
        search_TV = re.search('TV',list_ep[i])
        search_OVA = re.search('OVA',list_ep[i])
        search_Movie = re.search('Movie',list_ep[i])
        if search_TV:
#             a = list_ep[i].split('TV')[1]
            res.append(search_TV.group(0))
        elif search_OVA:
#             a = list_ep[i].split('OVA')[1]
            res.append(search_OVA.group(0))
        elif search_Movie:
#             a = list_ep[i].split('Movie')[1]
            res.append(search_Movie.group(0))
        else:
            res.append(np.nan)
            continue
    return res

        
    

In [ ]:
top_anime = []
top_anime_Rank = []
top_anime_Title = []
top_anime_Rating = []
top_anime_URL = []
top_anime_Image_URL = []
top_anime_Episodes = []
top_anime_Dates = []
top_anime_m_count = []
try_list = []
counter = 0

### inspierd by https://blog.jovian.ai/scraping-data-using-beautiful-soup-and-python-4170e7ec63fd ###
### inspierd by ###

for link in top_anime_list:
    
    ### Load the soup object trough load_soup_obj() ###
    html_get = requests.get(link)
    t_html_get = html_get.text
    counter+=1
    if html_get.status_code != 200:
            print('Status code:', html_get.status_code)
    
    print(counter)
    time.sleep(2)
    soup = BeautifulSoup(t_html_get,'lxml')
    soup.title.text.strip()
    
    ### find all Rank, Title, Rating, Date of Release, Episodes, Image_Url ###
    headers_mal = soup.find('tr', class_ = 'table-header')
    ### extract Rank, Title, Rating, Date of Release, Episodes, Image_Url from the given HTML ###
    row_content = soup.find_all('tr', {'class' : "ranking-list"})
    #print(row_content)

    for row in row_content:
        episode = parse_episodes(row.find('div', class_ = "information di-ib mt4").text.strip().split('\n'))

        
        
        top_anime_URL.append(row.find('td', class_= "title al va-t word-break").find('a')['href'])
        top_anime_Rank.append(row.find('td', class_ = "rank ac").find('span').text)
        top_anime_Title.append(row.find('div', class_="di-ib clearfix").find('a').text)
        top_anime_Rating.append(row.find('td', class_="score ac fs14").find('span').text)
        top_anime_Image_URL.append(row.find('td', class_ ='title al va-t word-break').find('img')['data-src'])
        top_anime_Episodes.append(episode[0])
        top_anime_Dates.append(episode[1]) 
        top_anime_m_count.append(episode[2])
        

        

top_anime_df = pd.DataFrame({'Rank':top_anime_Rank,'Title':top_anime_Title,
                                 'Rating':top_anime_Rating,'Image_URL':top_anime_Image_URL
                                 ,'Episodes':top_anime_Episodes,'Dates':top_anime_Dates,
                                'URL':top_anime_URL,'Members_Votes':top_anime_m_count},)
        



In [434]:

### First Handle of Data ###
copy = top_anime_df.copy()
list_dates = copy.Dates.to_list()
top_anime_df['Episodes'][0]
list_types = split_type(top_anime_df['Episodes'])


### Handle Data ###
copy['Type'] = list_types
copy['Members_Votes'] = delete_members_string(copy['Members_Votes'])
copy['Episodes'] = delete_ep_string(copy['Episodes'])
copy.set_index('Rank',inplace=True)
dic_dates = split_dates(list_dates)
copy['Start Date'] = dic_dates['Start']
copy['Finish Date'] = dic_dates['Finish']
copy.drop('Image_URL',axis=1, inplace=True)
copy.drop('Dates',axis=1, inplace=True)



In [435]:

read_copy = pd.read_csv('Anime_DataFrame_VA_1.csv',index_col='Rank')
read_copy



,Title,Rating,Episodes,URL,Members_Votes,Type,Start Date,Finish Date
Rank,,,,,,,,
1,Fullmetal Alchemist: Brotherhood,9.15,(64 eps),https://myanimelist.net/anime/5114/Fullmetal_A...,"2,720,933",TV,Apr 2009,Jul 2010
2,Gintama°,9.09,(51 eps),https://myanimelist.net/anime/28977/Gintama°,"495,687",TV,Apr 2015,Mar 2016
3,Shingeki no Kyojin Season 3 Part 2,9.09,(10 eps),https://myanimelist.net/anime/38524/Shingeki_n...,"1,650,249",TV,Apr 2019,Jul 2019
4,Steins;Gate,9.09,(24 eps),https://myanimelist.net/anime/9253/Steins_Gate,"2,122,353",TV,Apr 2011,Sep 2011
5,Fruits Basket: The Final,9.06,(13 eps),https://myanimelist.net/anime/42938/Fruits_Bas...,"293,483",TV,Apr 2021,Jun 2021
...,...,...,...,...,...,...,...,...
9996,Idol Project,5.63,(4 eps),https://myanimelist.net/anime/2072/Idol_Project,"1,883",OVA,Sep 1995,Oct 1997
9997,Kekkaishi,5.63,NaN,https://myanimelist.net/anime/40347/Kekkaishi,471,NaN,2004,2004
9998,Kodomo no Keijijougaku,5.63,NaN,https://myanimelist.net/anime/8333/Kodomo_no_K...,"1,022",NaN,Oct 2007,Oct 2007
